In [2]:
import pandas as pd
import numpy as np
import openpyxl as oxl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px

warnings.filterwarnings('ignore')


# Introduction

We will be working with t[IMD Digital Compeitiveness Ranking report](https://www.imd.org/centers/wcc/world-competitiveness-center/rankings/world-digital-competitiveness-ranking/) and compare the ranks against different countries using the data that has been provided in the report. The aim of this exercise is to use different visualisation techniques to understand the report when comparing the scores against multiple countries. 

The IMD digital competitiveness report provides tables around four main categories:
- Overall Digital Competitiveness
- Knowledge
- Technology
- Future Readiness

It also have an additional table covering sub-categories such as:
- Talent
- Training & education
- Scientific concentration
- Regulatory framework
- Capital
- Technological framework

In this report we will explore the digital competitive landscape for <font color="aqua">63 countries</font> selected by IMD and draw comparison analysis.

# Preprocessing

Lets begin by confirming the number of worksheets we have in the workbook:

In [16]:
#How many sheets  and observations are there in this workbook
workbook = oxl.load_workbook("../Data/IMD_digital_competencies.xlsx")

for sheet in workbook.worksheets:
    print(f'Sheet Name: {sheet.title} | No. Observations: {sheet.max_row -1}') 

Sheet Name: Overall | No. Observations: 63
Sheet Name: Knowledge | No. Observations: 63
Sheet Name: Technology | No. Observations: 63
Sheet Name: FutureReadiness | No. Observations: 63
Sheet Name: SubFactors | No. Observations: 63


As shown above, we can see that we have five worksheets each with the different categories, the last sheet SubFactors is theh sheet that represents the sub-category.

In [6]:

wb= "../Data/IMD_digital_competencies.xlsx"
df= pd.read_excel(wb, sheet_name='Overall')


,Country,2018,2019,2020,2021,2022
0,Argentina,55.0,59.0,59.0,61.0,59
1,Australia,13.0,14.0,15.0,20.0,14
2,Austria,15.0,20.0,17.0,16.0,18
3,Bahrain,NaN,NaN,NaN,NaN,32
4,Belgium,23.0,25.0,25.0,26.0,23


In [7]:
#We will impute these numbers with the rank 63, basically indicating that if the data has not been collected, their competitiveness is basically at last place
df.replace(np.nan,63, inplace=True)

Country    0
2018       2
2019       2
2020       2
2021       1
2022       0
dtype: int64

,Country,2018,2019,2020,2021,2022
0,Argentina,55.0,59.0,59.0,61.0,59
1,Australia,13.0,14.0,15.0,20.0,14
2,Austria,15.0,20.0,17.0,16.0,18
3,Bahrain,63.0,63.0,63.0,63.0,32
4,Belgium,23.0,25.0,25.0,26.0,23


In [13]:
#To visualise data differently, we are going to create scores rather than ranks

percentile_score = lambda x: (len(x) - x.rank() + 1) / len(x) * 100
df[['2018 Score', '2019 Score', '2020 Score', '2021 Score', '2022 Score']] = df.iloc[:, 1:].apply(percentile_score)
df.head()

,Country,2018,2019,2020,2021,2022,2018 Score,2019 Score,2020 Score,2021 Score,2022 Score
0,Argentina,55.0,59.0,59.0,61.0,59,15.873016,9.523810,11.111111,7.936508,7.936508
1,Australia,13.0,14.0,15.0,20.0,14,80.952381,79.365079,77.777778,69.841270,79.365079
2,Austria,15.0,20.0,17.0,16.0,18,77.777778,69.841270,74.603175,76.190476,73.015873
3,Bahrain,63.0,63.0,63.0,63.0,32,3.174603,3.174603,3.174603,3.968254,50.793651
4,Belgium,23.0,25.0,25.0,26.0,23,65.079365,61.904762,61.904762,60.317460,65.079365


In [159]:
#Create continent categories:
Asia_Pacific = ["Japan","China","India","Singapore","Indonesia","Malaysia","Hong Kong SAR", "Thailand", "Taiwan, China","Mongolia","Philippines","Korea Rep","Kazakhstan","Australia","New Zealand"]
Americas= ["Argentina","Brazil", "Venezuela","Canada","Chile","Colombia","USA","Mexico","Peru"]
Europe= ["Denmark","Norway","Iceland","Belgium","Austria","Germany","Italy","France","Netherlands","Portugal",
         "Sweden", "Croatia","Finland","Cyprus","Czech Republic","Estonia","Hungary","Latvia","Ireland","Luxembourg","Lithuania",
         "Greece","Slovak Republic","Spain","Switzerland","Slovenia","Turkey","Bulgaria", "Romania","Poland","United Kingdom"]
Middle_East_Africa= ["Saudi Arabia","Bahrain", "Israel", "UAE","Jordan","Qatar","Botswana","South Africa"]


#G7 and G10 categories
G7= ["USA","Canada","Japan","Germany","France", "United Kingdom","Italy"]
G10 =["USA","Canada","Japan","Germany","France", "United Kingdom","Italy", "Sweden","Belgium","Netherlands","Switzerland"]


In [160]:
#Apply continents and Geo categories
df["G7"] = np.where(df["Country"].isin(G7),1,0)
df["G10"] = np.where(df["Country"].isin(G10),1,0)
df["Continent"] = np.where(df["Country"].isin(Americas),"Americas",
                           np.where(df["Country"].isin(Europe),"Europe",
                                             np.where(df["Country"].isin(Middle_East_Africa),"Middle East & Africa",
                                                      np.where(df["Country"].isin(Asia_Pacific),"Asia & The Pacific","NA"))))
               

In [162]:
df=df.sort_values('2022 Score',ascending=False)
fig = px.bar(df, x='Country', y='2022 Score', title= 'Overall Digital Competitiveness Score by Country',color='Continent')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [166]:
df_Gp=pd.DataFrame(df.groupby('Continent')['2022 Score'].mean())
df_Gp.reset_index(inplace= True)
df_Gp.columns= ['Continent','2022 Average Score']
fig = px.bar(df_Gp, x='Continent', y='2022 Average Score', title= 'Average Digital Competitiveness Score by Continent',color='Continent')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [227]:
#Lets look at these scores from  a population perspective
population ="../Data/WB_Population_Data2021.csv"
pop = pd.read_csv(population, skiprows=4)
pop= pop[['Country Name','2021']]

pop['Country Name'].replace('United States','USA', inplace=True)
pop['Country Name'].replace('Venezuela, RB','Venezuela', inplace=True)
pop['Country Name'].replace('Korea, Rep.','Korea Rep', inplace=True)
pop['Country Name'].replace('Hong Kong SAR, China','Hong Kong SAR', inplace=True)
pop['Country Name'].replace('United Arab Emirates','UAE', inplace=True)
pop['Country Name'].replace('Czechia','Czech Republic', inplace=True)
pop['Country Name'].replace('Turkiye','Turkey', inplace=True)

#Filter and merge
df_pop=pd.merge(df,pop, left_on='Country',right_on= 'Country Name',how='left')

#Get population data from google for 2021 as it is missing from the pop data frame
df_pop.iloc[10,15] = 23855010.0

#Drop the Country Name column
df_pop.drop('Country Name',axis=1, inplace=True)
df_pop.head()


,Country,2018,2019,2020,2021,2022,2018 Score,2019 Score,2020 Score,2021 Score,2022 Score,G7,G10,Continent,2021
0,Denmark,4.0,4.0,3.0,4.0,1,95.238095,95.238095,96.825397,95.238095,100.000000,0,0,Europe,5856733.0
1,USA,1.0,1.0,1.0,1.0,2,100.000000,100.000000,100.000000,100.000000,98.412698,1,1,Americas,331893745.0
2,Sweden,3.0,3.0,4.0,3.0,3,96.825397,96.825397,95.238095,96.825397,96.825397,0,1,Europe,10415811.0
3,Singapore,2.0,2.0,2.0,5.0,4,98.412698,98.412698,98.412698,93.650794,95.238095,0,0,Asia & The Pacific,5453566.0
4,Switzerland,5.0,5.0,6.0,6.0,5,93.650794,93.650794,92.063492,92.063492,93.650794,0,1,Europe,8703405.0


In [229]:
pd.set_option('float_format', '{:f}'.format)
df_pop.groupby('Continent')['2021'].mean()

Continent
Americas                98878338.333333
Asia & The Pacific     238669328.800000
Europe                  19782822.000000
Middle East & Africa    16494999.625000
Name: 2021, dtype: float64

In [239]:
pop_mean= df_pop['2021'].mean()
pop_mean

82780515.17460318

In [236]:
df_pop['Above_Total_Pop_Mean']= np.where(df_pop['2021']>pop_mean,'Yes','No')

In [242]:
#It is not fair to look at it from an population angle
fig = px.bar(df_pop, x='Country', y='2021', title= 'Total Population By Country',color='Above_Total_Pop_Mean')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [241]:
fig = px.bar(df_pop, x='Country', y='2022 Score', title= 'Overall Digital Competitiveness Score by Country with total average population above 83 Million',color='Above_Total_Pop_Mean')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [246]:
#Create a normalised population column

df_pop['2021'].describe()


count           63.000000
mean      82780515.174603
std      250050425.073445
min         372520.000000
25%        5450406.500000
50%       11592952.000000
75%       51630719.000000
max     1412360000.000000
Name: 2021, dtype: float64

In [59]:
G7_Results= df[df["G7"]==1]
fig = px.bar(G7_Results, x='Country', y='2022 Score', title= 'Overall Digital Competitiveness Score by Country',color='Continent')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [3]:
# Let us take a look at the Technology Factors

wb= "../Data/IMD_digital_competencies.xlsx"
knowledge= pd.read_excel(wb, sheet_name='Knowledge')

In [5]:
#We will impute these numbers with the rank 63, basically indicating that if the data has not been collected, their competitiveness is basically at last place
knowledge.replace(np.nan,63, inplace=True)

In [6]:
#To visualise data differently, we are going to create scores rather than ranks

percentile_score = lambda x: (len(x) - x.rank() + 1) / len(x) * 100
knowledge[['2018 Score', '2019 Score', '2020 Score', '2021 Score', '2022 Score']] = knowledge.iloc[:, 1:].apply(percentile_score)
knowledge.head()

,Country,2018,2019,2020,2021,2022,2018 Score,2019 Score,2020 Score,2021 Score,2022 Score
0,Argentina,58.0,58.0,50.0,55.0,58,12.698413,12.698413,25.396825,17.460317,9.523810
1,Australia,15.0,15.0,17.0,19.0,14,77.777778,77.777778,74.603175,71.428571,79.365079
2,Austria,13.0,10.0,11.0,10.0,13,80.952381,85.714286,84.126984,85.714286,80.952381
3,Bahrain,63.0,63.0,63.0,63.0,34,3.174603,3.174603,3.174603,3.968254,47.619048
4,Belgium,25.0,23.0,21.0,21.0,21,63.492063,66.666667,68.253968,68.253968,68.253968


In [7]:
knowledge=knowledge.sort_values('2022 Score',ascending=False)
fig = px.bar(knowledge, x='Country', y='2022 Score', title= 'Digital Knowledge Score by Country')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [107]:
df1= df[['Country', '2018 Score','2019 Score','2020 Score','2021 Score','2022 Score']]
df1.rename(columns={'2018 Score':2018, '2019 Score':2019, '2020 Score':2020, '2021 Score':2021,'2022 Score':2022},inplace=True)
df1.head()
df1.set_index('Country', inplace=True)
df2=df1.T
df2.head()
df2.reset_index(inplace=True)
df2.rename(columns={'index':'Year'}, inplace=True)

df2['Year']= pd.to_datetime(df2['Year'],format='%Y')
df2['Year']= df2['Year'].astype('datetime64[Y]')

df2.head()

Country,Year,Denmark,USA,Sweden,Singapore,Switzerland,Netherlands,Finland,Korea Rep,Hong Kong SAR,...,Turkey,Mexico,Philippines,Peru,South Africa,Argentina,Colombia,Botswana,Mongolia,Venezuela
0,2018-01-01,95.238095,100.000000,96.825397,98.412698,93.650794,87.301587,90.476190,79.365079,84.126984,...,20.634921,22.222222,14.285714,9.523810,25.396825,15.873016,11.111111,3.174603,7.936508,3.174603
1,2019-01-01,95.238095,100.000000,96.825397,98.412698,93.650794,92.063492,90.476190,85.714286,88.888889,...,20.634921,25.396825,15.873016,7.936508,26.984127,9.523810,11.111111,3.174603,6.349206,3.174603
2,2020-01-01,96.825397,100.000000,95.238095,98.412698,92.063492,90.476190,85.714286,88.888889,93.650794,...,33.333333,17.460317,12.698413,15.873016,9.523810,11.111111,7.936508,3.174603,6.349206,3.174603
3,2021-01-01,95.238095,100.000000,96.825397,93.650794,92.063492,90.476190,84.126984,82.539683,98.412698,...,26.984127,15.873016,12.698413,14.285714,9.523810,7.936508,11.111111,3.968254,6.349206,1.587302
4,2022-01-01,100.000000,98.412698,96.825397,95.238095,93.650794,92.063492,90.476190,88.888889,87.301587,...,15.873016,14.285714,12.698413,11.111111,9.523810,7.936508,6.349206,4.761905,3.174603,1.587302


In [120]:
fig = px.line(df2, x="Year", y=['Canada', 'USA', 'Denmark','Sweden','Singapore','Finland','Korea Rep','Hong Kong SAR','Switzerland','Netherlands'], title= 'Top 10 Rank Countries over time')
fig.show()

In [121]:
fig = px.line(df2, x="Year", y=['Japan', 'Philippines', 'Malaysia','China','Singapore','India','Korea Rep','Hong Kong SAR'], title= 'Top 10 Rank Countries over time')
fig.show()